In [3]:
#colab

# from google.colab import drive
# drive.mount('/content/gdrive')

# import sys
# sys.path.append('/content/gdrive/MyDrive/ajou_ribs/img')

import numpy as np
import warnings
import matplotlib.pyplot as plt
import matplotlib.patches as patches
warnings.filterwarnings(action='ignore')

---
# 수학과 201521139 이재학
### Keywords : Frechet distance, interpoint distance
---
### 진행 방식 참고:
<img src="허권논문.jpg" width="300px" height="300px">


# Contents
- 논문소개   
- 배경지식
  - Hausdorff distance (21.10.27 진행) 
  - frechet distance
  - discrete frechet distance (21.11.10 진행)
- 활용
  - 다차원프레셰 거리 기반 종단자료 군집분석(MFKmL)
  - 대립생성망의 성능 비교에 대한 연구 (FID)
  - 이산 프레셰 거리 척도를 이용한 궤적 유사도 고속계산 휴리스틱
- 논문분석
- 참고

---
# 논문 : [computing the frechet distance between polygonal curves](https://www.worldscientific.com/doi/abs/10.1142/S0218195995000064)    
<br/>
<img src="fd.png" width="400px" height="400px">
<br/> 
- 초록
  - 임의의 차원에서 곡선의 유사성에 대한 측정을 하기 위해, 곡선의 매개변수화가 가능한 프레셰 거리를 고려한다.<br/>
  - 간선이 각 p,q개인 다각형 P,Q의 프레셰 거리는 O(pqlog(pq))의 runtime으로 구할 수 있다.<br/>
  - 더 나아가, 닫힌 곡선에 대한 프레셰 거리, 비단조 프레셰거리 그리고 기준이 되는 곡선 P가 다른 곡선 Q의 '일부'와의 유사성을 측정하는 프레셰 거리에 대한 변화를 고려해보려한다. 
  
- 서론
  - 실생활에서, 주어진 두 곡선의 거리는 '직관적으로' 두 곡선이 얼마나 '유사한지'로 표현된다.
  - 측정하는 거리로 '하우스 도르프 거리'가 제안된다.

---
# 배경지식
- 1.Hausdorff distance
  - 정의 :
<br/>
<img src="ha_dis.jpg" width="400px" height="400px">
<br/> 
    - 점으로 이루어진 두 집합(point sets) 간의 거리를 결정하는 방법
    
<br/> 

  - 예시 :
<br/>
<img src="hd_ex.jpg" width="500px" height="500px">
<br/>
    - 두 집합 사이의 근접점에서 떨어진 가장 먼 지점을 찾음
<br/>
    - 직관적으로, 한쪽 점 집합을 기준으로 다른쪽 집합 상의 점까지의 가장 먼 거리를 구함. 그리고 두 거리중 더 큰 것을 distance로 정함.
<br/>   
<br/>
  - 단점 :
<br/>
<img src="hd_단점.jpg" width="400px" height="400px">
<br/>
    - outlier에 취약
<br/>
    - outlier (x4,y4)을 제외하면 hausdorff 거리는 왼쪽이 훨씬 작음.
<br/>
<img src="논문단점.jpg" width="500px" height="500px">
<br/>
    - 논문에서 제기한 단점 : 두 궤적의 진행 방향이나 모양을 고려하지 않기 때문에  실제 두 궤적의 유사성을 판단하기엔 부적합
    
<br/>

  - 활용1 : 컴퓨터 비전 분야에서 주로 쓰이며, '매칭' 문제 해결을 위해 사용됩니다.
<br/>

<center>OpenCV 에서의 Template Matching</center>

<br/>
<img src="템플릿매칭.jpg" width="500px" height="500px">
<br/>

    - Template Matching : 영상에서 작은 크기의 템플릿 영상과 일치하는 부분을 찾는 기법  
    - 한 템플릿이 기준이기에 속도가 매우 느림  
    - Hausdorff distance를 이용하여, 각 템플릿에 대한 최소의 하우스 도르프 거리를 갖는 이미지 영역은 템플릿을 찾는데 가장 적합한 후보    
  - 활용2 : PostGIS(Geospatial 데이터를 다루는 SQL)
    - PostGIS에서 공간 쿼리를 다룰 때, 공간 위상관계에서 교차하는 객체를 찾아 객체간 거리 산출
    - 거리값이 작을수록 두 건물은 유사한 형상  
    - 거리 산출 수행: ST_HausdorffDistance 함수 실행
<br/>
<img src="postgis.jpg" width="600px" height="600px">
<br/>
  - 활용3 : Python Scipy Library
<br/>
<img src="scipy.jpg" width="600px" height="600px">
<br/>

In [4]:
from scipy.spatial.distance import directed_hausdorff
u = np.array([(1.0, 0.0),
              (0.0, 1.0),
              (-1.0, 0.0),
              (0.0, -1.0)])
v = np.array([(2.0, 0.0),
              (0.0, 2.0),
              (-2.0, 0.0),
              (0.0, -4.0)])

print(directed_hausdorff(u, v))
print(directed_hausdorff(v, u))

(2.23606797749979, 3, 0)
(3.0, 3, 3)


<br/>
<img src="hdextap.jpg" width="800px" height="800px">
<br/>

---
- 2.Frechet distance
  - 정의 : @@@이미지넣기@@@  
  - 예시 :  
  - 유래 : 수학에서 곡선을 따라 점의 위치와 순서를 고려한 곡선 간의 유사도를 측정 한 것. '모리스 르네 프레셰'의 이름을 따서 명명됨
- 3.Discrete Frechet distance

# TODO
- frechet distance ,discrete frechet distance 의 소개 및 활용,정의,예시 -> 서론 그림 밑 글부터
---

- multinomial 논문 +유튜브참고(시간복잡도)  
- 프레셔 vs interpoint  
- FID -> 인셉션 모델이란? / FID 파이썬코드
- R이나 파이썬으로 구현?

---
# <참고>

[위키피디아](https://en.wikipedia.org/wiki/Fr%C3%A9chet_distance)  
[유튜브 : Frechet Distance Between Two Point Sets](https://www.youtube.com/watch?v=12vrDDBnEFg)  
[유튜브 : GAN 성능의 정량적 평가 방법 - Python, Deep Learning](https://www.youtube.com/watch?v=19An2T4utEM)


# <논문>
### 메인
[computing the frechet distance between polygonal curves](https://www.worldscientific.com/doi/abs/10.1142/S0218195995000064)
### 참고
[이산 프레셰 거리 척도를 이용한 궤적 유사도 고속계산 휴리스틱](http://www.dbpia.co.kr.ssl.openlink.ajou.ac.kr/search/topSearch?startCount=0&collection=ALL&range=A&searchField=ALL&sort=RANK&query=%EC%9D%B4%EC%82%B0+%ED%94%84%EB%A0%88%EC%85%B0&srchOption=*&includeAr=false)  
[다차원 프레셰 거리 기반 종단자료 군집분석](https://dcoll.ajou.ac.kr/dcollection/srch/srchDetail/000000030579)  
[대립생성망의 성능 비교에 대한 연구](https://www.dbpia.co.kr/Journal/articleDetail?nodeId=NODE07540262)  
[The Frechet Distance between Multivariate Normal Distributions](https://www.sciencedirect.com/science/article/pii/0047259X8290077X)  
[Interpoint distances: Applications, properties, and visualization](https://onlinelibrary.wiley.com/doi/abs/10.1002/asmb.2508)

# <개념>
#### Hausdorff distance
[Hausdorff distance](https://progworks.tistory.com/72)  
[Hausdorff distance 개념](https://dhpark1212.tistory.com/entry/Hausdorff-Distance)  
[Template Matching](https://velog.io/@codren/%ED%85%9C%ED%94%8C%EB%A6%BF-%EB%A7%A4%EC%B9%AD)

---
# 대본참고  
- 하려고하는배경(목적) / 해결하기 위한 방법론(장단점극복) /새로운 방법 제안  
- 발표 10분
---



# 대본 

(진행방식참고)
안녕하세요, 문헌조사 및 논문정리에서  frechet distance와 interpoint distance를 키워드로 발표를 하게된 수학과 15학번 이재학입니다.
발표를 진행하는데에 있어, 진행방식은 권순선교수님이 지도교수로 계시고, 저희의 선배이신 존경스러운 허성보님의 석사학위 논문의 진행방식을 참고하였습니다. 논문을 찾게 된 계기는 제가 맡은 키워드로 논문을 쓰셔서 참고하던 중 발견했습니다.
그리고 ppt로 진행하는게 맞지만, 업데이트가 계속 될 예정이고,실습을 r이나 파이썬으로 진행해보기 위해 주피터 마크다운문서로 작성했습니다. 마무리 지을때쯤 시간의 여유가 있다면 ppt로만들어보겠습니다.

(contents)
논문분석 진행 컨텐츠는, 논문소개 그리고 배경지식소개,활용소개,분석소개,참고사항 등으로 진행될예정이며,
논문분석이 후순서로 진행되는 이유는, 논문의 양이 굉장히 방대하고,배경지식이 전무한 상태로 분석이 진행됨에따라, 배경지식의 설명이 중요할것같아서,이렇게 진행하게됐습니다. 금일은 논문의 간단한 소개와 배경지식의 첫번째인 하우스도르프 거리 설명 및 예시까지 진행할것입니다.
(논문소개)
논문을 읽을때는 권교수님께서 해주셨던 조언을따라, 하려고하는 배경및목적, 해결하기위한 방법론과 장단점극복 그리고 새로운방법 제안의 서술을따라 읽어보려고 노력했고, 따라서 기본이 되는 이 논문을 설정했습니다. 인용수가 가장 많았고 오래된 논문이기도 합니다.

우선,제 논문의 키워드는 프레셰거리로,  보시는 그림처럼 곡선의 거리를 측정을 하는 것을 목적으로 합니다.
논문의 초록부터 살펴봤을때,해당논문은 임의의 차원에서 곡선의 유사성에 대해 측정을하기위해 곡선의 매개변수화가 가능한 프레셰 거리를 고려하며,
간선의 각 p,q개의 다각형 P,Q의 프레셰 거리는 빅오 노테이션의 log(pq)곱하기 pq의 런타임으로 구할 수 있습니다.
해당 런타임의 경우는 교수님이 해주셨던 조언에서 런타임을 해결하기위한 방법론으로 논문의 중반부분에 서술되어 추후에 설명드릴 예정입니다.
그리고, 프레셰거리를 닫힌곡선,논모노톤,그리고 곡선의 부분집합에서 측정하는 프레셰거리또한 논문의 중반이후에 서술되어있기에 본론부분에서 설명드릴 예정입니다. 

다음은 논문의 서론부분입니다.논문의 서론의 초반부에서는, 프레셰거리와 유사성에 대해 서술되어있습니다.
실생활에서, 두 곡선의 거리는 직관적으로 두 곡선이 얼마나 유사한지로 표현되며, 측정하는 거리로 하우스 도르프 거리가 제안되는데, 이의 단점을 보완하기 위함으로
프레셰 거리가 제안된다. 라는 일반적인 전개로 이루어집니다.금일은 하우스 도르프거리의 설명까지 할 예정이므로, 서론의 초반부까지 마치고 배경지식으로 넘어가겠습니다.

(배경지식)
하우스도르프 거리는 점으로 이루어진 두 집합 간의 거리를 결정하는 방법으로, 정의는 보시는 수식과 같습니다.
 예시로 보여지는 그림과 정의를 참고하면, 두 집합 사이의 근접점에서 떨어진 가장 먼 지점을 찾는것으로 이해할 수 있으며,
직관적으로 살펴보면, 한쪽 점 집합을 기준으로 다른쪽 집합 상의 점까지의 가장 먼거리를 구하고, 두 거리중 더 큰것을 하우스도르프 거리로 정하게됩니다.
이부분에서 이해가안되셔도 걱정하지않으셔도됩니다.마지막부분에 제가 직접그린그림으로 다시 설명드리겠습니다.

(단점)
이러한 하우스도르프 거리는 단점이존재합니다. 우선첫번째로 아웃라이어에 대해 취약하다는것인데.
보시는 흑백그림에서 하우스도르프거리는 왼쪽이 더 크게 나오는데,사실 outlier인 x4컴마y4를 제외한다면 오히려 거리가 훨씬작게됩니다.
두번째는 논문에서 제기한 단점으로,두 궤적의 진행 방향이나 모양을 고려하지 않기에 실제 두 궤적의 유사성을 판단하기에는 부적합하다는 의견입니다.
지그재그로 표시된 그림은 논문에 있는 그림인데, 정의에 따르면 인접한점들로인해 거리가 적게나오지만, 유사성으로 접근했을경우 상반된 결과를 나타내게됩니다.
이는,하우스도르프거리가 두 곡선의 접 집합만 고려하며,곡선의 진행 방향을 고려하지 않기에 문제가 생기는 것입니다.
따라서,이를 해결하기위해 서론에서는 프레셰 거리가 제안됩니다.

(활용1)
다음은 프레셰거리를 다루기전에 ,하우스도르프거리에대한 활용입니다.

첫번째는, 컴퓨터비젼분야에서 매칭 문제 해결을 위해 사용됩니다. 학부인턴분들중에는 컴퓨터를 복수전공하시는분들이 많기에,
opencv를 다뤄본분들이 많을것같은데, 저같은경우 원리도 모르고 사용했던 template matching에 하우스도르프거리가 사용됨을 알게되었습니다.
템플릿 매칭은 입력영상에서 작은 크기의 템플릿 영상과 일치하는 부분을 찾는기법으로, 예시에 해당하는 템플릿은 눈이고, 여성의 사진을 돌아다니며 눈을 찾는 예시입니다.
하지만,템플릿 매칭은 한템플릿이 기준이므로 속도가 느린 단점이있고, 여기서는 하우스도르프 거리를 이용하여, 각 템플릿에 대해 최소 하우스 도르프 거리를 갖는
이미지 영역이 템플릿을 찾는데에 가장 적합한 후보로 간주되는데에 사용이됩니다.

(활용2)
다음으로는 지오스패이셜 데이터를 다루는 sql인 postgis에서 사용됩니다. 공간 쿼리를 다룰 때,공간 위상관계에서 교차하는 객체를 찾아 객체간 거리를 산출하고,
거리값이 작을수록 두 건물은 유사한 형상을 띕니다. 밑에 보시는 사진은 다른기준으로 측정된 건물들을 중첩했을때 일치하지 않을경우,
postgis에서 하우스도르프거리를 사용하는 함수인 ST_하우스도르프디스턴스 함수를 실행했을때, 두건물간의 거리를 산출하여 비교할 수 있습니다.
데이터베이스를 현재 수강하고있기에 잘 모르는 분야라 넘어가겠습니다.

(활용3)
다음은 실습할 수 있는 예시로 파이썬의 사이파이 라이브러리를 가져왔습니다.
국가수리과학연구소에서 학부연수생을 할때, 코로나19를 예측하는 수리모델링에서 오일러 메쏘드를 풀기위해
미분장정식을 해결해주는 목적으로 사용했었는데,하우스도르프거리도 이 라이브러리에 있는것을 확인했습니다.
도큐먼트를 보시면, 리턴값으로, 거리,사용되는점의인덱스 두개가나옵니다.
(코드)
이를참고하여 코드를보겠습니다.엔디어레이에 두개의 좌표를 저장하고, 거리를 구해보면,
첫번째는 2.23 그리고 인덱스는 3,0이나옵니다. 이를 제가 태블릿에 그린 그림으로 매칭시켜보면,
하우스도르프 u,v는 우선 u,v의 근접점을 (-1,0)과 (-2,0)의 기준으로잡고, max값이 씌우기위해 멀어지는
v의 좌표인 (0,2)이 설정되며, 결국 (-1,0)과(0,2)사이의 거리인 루트5가나오고 그값은 앞에나왔던 2.23이나오게됩니다.
비슷하게 두번째로는 거리는 3 인덱스는 3과3이나오고,보시는 예와 같이 맞게나오는것을 확인할 수 있습니다.

이 부분까지가 금일 준비한 발표이며,다음발표때는 배경지식의 다음부분인 프레셰거리, 이산프레셰거리
소개 및 활용부터 시작될 예정입니다.

(참고부분)
해당부분은 발표를 준비함에 있어서 살펴본 자료와 논문들입니다.
이상 발표를 마치겠습니다 .감사합니다.
